# Exercise Sheet \# 7 - Interfacing Python with SQL databases

In this exercises, you are asked to design python code which can interact with a SQLite database. We will use again the [SQL database](http://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip) from Exercise Sheet \#6. 

## Exercise 1

Write a python script, which covers questions 1.5 to 1.9 from Exercise Sheet \#5, using the SQLalchemy API.

## Exercise 2
Extend your program so that it also covers questions 2.5, 2.6 and 2.8 from previous Exercise Sheet.